# Prepare input for escoring (expression matrix)

#### Note that we are in Python

#### Working directory should contain root atlas Seurat object, which is aviable on GEO under record GSE152766:

    Root_Atlas.rds (or get it by running through notebook 2)

In [1]:
import psutil
psutil.virtual_memory()

svmem(total=1082180472832, available=596283752448, percent=44.9, used=480730210304, free=437166096384, active=452687314944, inactive=14842261504, buffers=5361664, cached=164278804480, shared=3707510784, slab=159720357888)

In [2]:
#import scrublet as scr
import scipy.io
import os
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
from matplotlib.colors import ListedColormap
import seaborn as sb
from gprofiler import GProfiler
from scipy import sparse
import scvelo as scv
scv.logging.print_version()

from rpy2.robjects.packages import importr
seuratLib = importr('Seurat')

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

import h5py
import anndata
import warnings
warnings.filterwarnings('ignore')

#from SAM import SAM
#from mnnpy import mnn_correct as mnn_cor
import utilities as ut
import inspect

%matplotlib inline

Running scvelo 0.1.25 (python 3.7.3) on 2020-05-03 17:03.


In [3]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output

rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(15,15) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

scanpy==1.4.4.post1 anndata==0.6.22.post1 umap==0.3.9 numpy==1.17.3 scipy==1.3.2 pandas==0.25.0 scikit-learn==0.22 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1


In [4]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.set_figure_params('scvelo')  # for beautified visualization

In [5]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths(.libPaths()[c(3,2,1)])

library(RColorBrewer)
library(ggplot2)
library(plyr)
library(Seurat)
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /fast/home/c/chsu/anaconda3/envs/cbpsc/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.utf-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.utf-8        LC_COLLATE=en_US.utf-8    
 [5] LC_MONETARY=en_US.utf-8    LC_MESSAGES=en_US.utf-8   
 [7] LC_PAPER=en_US.utf-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.utf-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] plyr_1.8.4         ggplot2_3.2.1      RColorBrewer_1.1-2 Seurat_3.0.2      

loaded via a namespace (and not attached):
 [1] httr_1.4.1          tidyr_1.0.0         viridisLite_0.3.0  
 [4] jsonlite_1.6        splines_3.6.1       lsei_1.2-0         
 [7] R.utils_2.9.1       g

In [237]:
%%R -o normalized_counts

rc.integrated <- readRDS('./Root_Atlas.rds')
normalized_counts <- as.matrix(rc.integrated@assays$integrated@data)

In [238]:
normalized_counts = sparse.csr_matrix(normalized_counts, dtype=np.float32)

In [239]:
normalized_counts.shape

(17513, 110427)

In [243]:
# Load modules
from escoring.support_funcs import load_sparse_h5, save_sparse_h5

In [244]:
# Save the expression matrix as h5 format
save_sparse_h5(normalized_counts, 'counts', "./supp_data/hvg_integrated.h5")